In [79]:
!python3 -m pip install jmespath


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip


In [83]:
!python3 -m pip install pandas



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip


In [156]:
import json
import os
import sys

#### Desired data pairs and metadata

In [157]:
import jmespath
import json
import os
import sys

# Load the JSON file
def load_json_file(file_path):
    if not os.path.exists(file_path):
        print(f"File {file_path} does not exist.")
        sys.exit(1)

    with open(file_path, 'r') as file:
        try:
            data = json.load(file)
            return data
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON from file {file_path}: {e}")
            sys.exit(1)

# Parse a single Instagram post (items[0]) using jmespath
def parse_post_json(data: dict) -> dict:
    return jmespath.search(
        """{
            shortcode: code,
            caption: caption.text,
            head_image_url: image_versions2.candidates[0].url,
            audio_type: music_metadata.audio_type,
            music_title: music_metadata.music_info.music_asset_info.title,
            music_artist: music_metadata.music_info.music_asset_info.display_artist
        }""",
        data,
    )

# Extract all parsed posts from a combined user-post JSON
def extract_all_posts(json_data: dict):
    parsed_posts = []

    for username, posts_dict in json_data.items():
        for post_url, post_data in posts_dict.items():
            items = post_data.get("items", [])
            if items:  # Only if not empty
                parsed = parse_post_json(items[0])
                if parsed:
                    parsed["username"] = username  # Add source username
                    parsed_posts.append(parsed)

    return parsed_posts


In [158]:
# Load file
raw_json_data = load_json_file("all_instagram_data.json")

# Parse all valid posts
parsed_posts = extract_all_posts(raw_json_data)

# Check result
for post in parsed_posts[:3]:  # print first 3 for preview
    print(json.dumps(post, indent=2))


{
  "shortcode": "DIe1T1bg2Wf",
  "caption": "Keep pushing \ud83d\udfe1\ud83d\udd35",
  "head_image_url": "https://scontent-lax3-1.cdninstagram.com/v/t51.2885-15/490522047_18618032941056421_8208559749308424692_n.jpg?stp=dst-jpg_e35_p1080x1080_tt6&cb=30a688f7-fa102a98&efg=eyJ2ZW5jb2RlX3RhZyI6IkNBUk9VU0VMX0lURU0uaW1hZ2VfdXJsZ2VuLjE0NDB4MTgwMC5zZHIuZjc1NzYxLmRlZmF1bHRfaW1hZ2UifQ&_nc_ht=scontent-lax3-1.cdninstagram.com&_nc_cat=1&_nc_oc=Q6cZ2QEfdLRJw3RndhUo_Z1XEoJUSoztgM7i94a4oAVOqpq0b7YkYdynQMf9b6sMyIhkX4U&_nc_ohc=_RRrN27-2iUQ7kNvwFDmhq_&_nc_gid=1ksWLzgdqbPLZr3pcB0QJA&edm=ALQROFkBAAAA&ccb=7-5&ig_cache_key=MzYxMTU1ODQwMzAxODYwODE4Nw%3D%3D.3-ccb7-5-cb30a688f7-fa102a98&oh=00_AfH6N5wgU1LY7k9eGCzI3MXi_83zv_yg7Ob56UECBCvCIg&oe=6804E621&_nc_sid=fc8dfb",
  "audio_type": null,
  "music_title": null,
  "music_artist": null,
  "username": "cristiano"
}
{
  "shortcode": "DIXBP7gA9uA",
  "caption": "This is how we do it! \u26bd\ufe0f\u26bd\ufe0f",
  "head_image_url": "https://scontent-lax3-1.cdninstagr

In [159]:
import pandas as pd

df = pd.DataFrame(parsed_posts)
df.head()

,shortcode,caption,head_image_url,audio_type,music_title,music_artist,username
0,DIe1T1bg2Wf,Keep pushing 🟡🔵,https://scontent-lax3-1.cdninstagram.com/v/t51...,None,None,None,cristiano
1,DIXBP7gA9uA,This is how we do it! ⚽️⚽️,https://scontent-lax3-1.cdninstagram.com/v/t51...,None,None,None,cristiano
2,DIL-t6qoUNX,Um dos grandes símbolos da formação mundial de...,https://scontent-lax3-1.cdninstagram.com/v/t51...,None,None,None,cristiano
3,DIKJQTdgCtD,Hard work is the only way to success. 🔥,https://scontent-lax3-1.cdninstagram.com/v/t51...,None,None,None,cristiano
4,DICffjKAj3o,RIYADH IS YELLOW AND BLUE!,https://scontent-lax3-1.cdninstagram.com/v/t51...,None,None,None,cristiano


In [160]:
# for all "None" on audio type, remove them
df = df[df['audio_type'] == 'licensed_music']

# remove id, caption, audio type, 
df = df.drop(columns=['username', 'caption', 'audio_type'])

df

,shortcode,head_image_url,music_title,music_artist
81,DHqyEKlx45m,https://scontent-lax3-1.cdninstagram.com/v/t51...,Call Me When You Break Up,"Selena Gomez, benny blanco, Gracie Abrams"
92,DHKpuJEtril,https://scontent-lax3-2.cdninstagram.com/v/t51...,Sunset Blvd,"Selena Gomez, benny blanco"
104,C6bGvvzvQma,https://scontent-lax3-1.cdninstagram.com/v/t51...,Perfect,Ed Sheeran
107,C0vRPnvoIAa,https://scontent-lax3-1.cdninstagram.com/v/t51...,Those Eyes,New West
165,C_gUrFoSjoi,https://scontent-lax3-1.cdninstagram.com/v/t51...,I've Been In Love (feat. Channel Tres),Jungle
181,DIJGGzIp_M9,https://scontent-lax3-1.cdninstagram.com/v/t51...,FLOWER,JISOO
183,DH_AVl6J4-3,https://scontent-lax3-1.cdninstagram.com/v/t51...,earthquake,JISOO
185,DH1Ch34zp_-,https://scontent-lax3-1.cdninstagram.com/v/t51...,All Eyes On Me,JISOO
186,DHyhk84JaDA,https://scontent-lax3-1.cdninstagram.com/v/t51...,Your Love,JISOO
188,DHgcRhEJA60,https://scontent-lax3-1.cdninstagram.com/v/t51...,Annie (2022 Live Version),Yoon Jong Shin


In [161]:
#save df to csv
df.to_csv('all_instagram_data.csv', index=False)